In [1]:
from bcb import Expectativas
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
import os
from datetime import datetime, timedelta
from statsmodels.tsa.arima.model import ARIMA
import warnings
# Suprimir temporariamente os avisos
warnings.filterwarnings("ignore")

In [2]:
def Func_Expec_Selic():
    # Importando os dados do BCB
    # Instancia a classe
    em = Expectativas()

    SerieCaptada = "ExpectativasMercadoSelic"
    ep = em.get_endpoint(SerieCaptada)


    # Dados da Selic
    selic_expec = ( ep.query()
    .filter(ep.Indicador == 'Selic')
    .filter(ep.baseCalculo=='0')
    .orderby('Data asc')
    .select(ep.Indicador, ep.Data, ep.Reuniao,ep.Media, ep.Mediana, ep.DesvioPadrao, ep.baseCalculo)
    .collect()
    )

    def calcular_data_reuniao(reuniao):
        # Extrai o número da reunião e o ano
        numero_reuniao, ano = map(int, reuniao[1:].split('/'))
        
        # Calcula a data da primeira reunião do ano
        primeira_reuniao_data = datetime(ano, 2, 1)  # Supondo que a primeira reunião seja em 1 de fevereiro
        
        # Calcula a data da reunião com base no número da reunião e a diferença de dias
        dias_reuniao = (numero_reuniao - 1) * 45
        data_reuniao = primeira_reuniao_data + timedelta(days=dias_reuniao)
        
        return data_reuniao

    # Exemplo de uso
    reuniao = "R2/2022"
    data = calcular_data_reuniao(reuniao)
    # print(data)

    df_filtrado2 = selic_expec.copy()
    df_filtrado2["Reuniao"] = df_filtrado2["Reuniao"].apply(calcular_data_reuniao)
    df_filtrado2['Diferenca'] = (df_filtrado2['Reuniao'] - df_filtrado2['Data']).dt.days
    df_filtrado2 = df_filtrado2[df_filtrado2['Data']>datetime(2006,1,1)]

    grupos = df_filtrado2.groupby("Data")
    dic_selic = {}
    Selic_12Meses = pd.DataFrame(columns=["Valor"])
    for data, grupo in grupos:
        
        #print(data, grupo)

        df_temp = grupo.loc[:,["Data","Mediana","Diferenca"]]
        
        # Crie uma função de interpolação usando interp1d
        interpolacao = interp1d(df_temp['Diferenca'], df_temp['Mediana'], fill_value="extrapolate")   
        valor_interpolacao = np.round(interpolacao(365),2)
        df_temp.loc[df_temp.index[-1]+1,:] = [data, valor_interpolacao, 365]
        dic_selic[data] = df_temp
        Selic_12Meses.loc[data,"Valor"] = valor_interpolacao


    ## Colunas do ARIMA
    for i in range(1,13):
        Selic_12Meses[f"{i}_mes"] = 0

    return Selic_12Meses

In [3]:
def SELIC_ARIMA(df_Expectativa_Selic_mensal, n_steps = 12):
    (p, d, q) = (1,1,1)
    model = ARIMA(df_Expectativa_Selic_mensal, order=(p, d, q))
    model_fit = model.fit()
    # Faça previsões para os próximos n_steps meses
    forecast = model_fit.forecast(steps=n_steps)
    prev_n_steps_meses = forecast[-1]
    return forecast

In [4]:
Selic_12Meses = Func_Expec_Selic()

In [5]:
lista_index = Selic_12Meses.index
for data in lista_index[100:]:
    df_mensal = Selic_12Meses.loc[:data,"Valor"].resample('M').mean()
    serie = list(df_mensal.values)
    forecast = SELIC_ARIMA(serie)
    Selic_12Meses.loc[data,"1_mes":"12_mes"] = forecast

In [ ]:
nome_arquivo = f"Expectativa_Selic_Diaria.parquet"
arquivo_Selic = os.path.join("..","..","..","dataset", "BR","Selic", nome_arquivo)
# Substitua os valores infinitos por NaN (ou qualquer outro valor desejado)
Selic_12Meses.replace([np.inf, -np.inf], np.nan, inplace=True)
Selic_12Meses.to_parquet(arquivo_Selic, engine='fastparquet')